In [ ]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [ ]:
import pandas as pd
import pyspark
from pyspark.sql import *

In [ ]:
from pyspark.ml.feature  import RegexTokenizer,StopWordsRemover,CountVectorizer,OneHotEncoderEstimator,StringIndexer
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as f
from pyspark.sql.types import *


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [ ]:
sc=pyspark.SparkContext()

In [ ]:
spark= SparkSession \
       .builder \
       .appName("DICPA3") \
       .config("spark.some.config.option","some-value") \
       .getOrCreate() 

In [ ]:
train_pd = pd.read_csv('train.csv')

In [ ]:
mapping_pd = pd.read_csv('mapping.csv')


In [ ]:
mapping=spark.createDataFrame(mapping_pd)


In [ ]:
train=spark.createDataFrame(train_pd)

In [ ]:
train.show()

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")


train=regexTokenizer.transform(train)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

train=stopwordsRemover.transform(train)




In [ ]:
countvectors = CountVectorizer(inputCol="filtered", outputCol="features",vocabSize=1500)

In [ ]:
model1=countvectors.fit(train)

In [ ]:
train=model1.transform(train)

In [ ]:
mapping=spark.createDataFrame(mapping_pd,['id',"genre"])
mapgenre=mapping.select("genre","id").rdd.collectAsMap()

In [ ]:
mapping.show()
count=mapping.count()

In [ ]:

train.show()


In [ ]:

def function1(g):
    x=[]
    genre=g[1:-1].split(",")  
    for i in genre:
        value=i.strip()[1:-1]
        x.append(mapgenre.get(value))
    x.sort()
    return x
myfunction1=f.udf(function1,ArrayType(IntegerType()))
train=train.withColumn("index",myfunction1("genre"))

train.select('index').show()

In [ ]:

def function2(g):
    y=1
    x=[]
    for i in range(0,count):
        x.append(0)
    
    
    for i in g:
        x[i]=1
        
    
    return x
myfunction2=f.udf(function2,ArrayType(IntegerType()))
traindata=train.withColumn("label",myfunction2("index"))
traindata.select('movie_id','features','index','label').show()

In [ ]:
test_pd = pd.read_csv('test.csv')

In [ ]:
test=spark.createDataFrame(test_pd)

In [ ]:
test.show()

In [ ]:
test=regexTokenizer.transform(test)

test=stopwordsRemover.transform(test)

In [ ]:
test=model1.transform(test)

In [ ]:
test.show()

In [ ]:
x=[]

for i in range(0,count):
    
#     def parsePoint(line):
    
#         return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
#     parsedData = traindata.rdd.map(parsePoint)
#     part1model=LogisticRegressionWithLBFGS.train(parsedData)
    filename='./part1test'+str(i)
#     part1model.save(spark,filename)
    newmodel = LogisticRegressionModel.load(spark,filename)
    labelsAndPreds = test.rdd.map(lambda p: (p.movie_id, newmodel.predict(MLLibVectors.fromML(p.features))))
    x.append(labelsAndPreds.collect())

In [ ]:
y={}
for i in x:
    for j in i:
        if j[0] in y:
            y[j[0]].append(j[1])
        else:
            y[j[0]]=[]
            y[j[0]].append(j[1])
for i in y:
    print(i,y[i])

In [ ]:
keypred={}

for i in y:
    x=""
    for j in y[i]:
        x=x+" "+str(j)
    x=x.strip()
    keypred[i]=x
for i in keypred:
    print(i,str(keypred[i]))

In [ ]:
import csv

with open ('output1.csv','w',newline="") as mfile:
    writer=csv.writer(mfile)
    writer.writerow(["movie_id","predictions"])
    for key , value in keypred.items():
        writer.writerow([key,value])